# Landslide Hazard Analysis Using SAR
Intro to the use case/methods used

Search for SLC images over Haiti in a defined time period

Request HyP3 processing of Haiti data for two years

Download the staged version of the data

Crop all images to same extent (e.g. InSAR Notebook)

Load the geotiffs into Xarray with datetime stamps for each image

Pre-temporal averaging, create time series plot of single pixel (or group of pixels)

Create a temporal average for both pre and post EQ

Perform a log difference of the two resulting images.

Discuss options for thresholding the difference image

Use the COP30 DEM to remove change detections in areas with slopes <5 degrees use gdaldem

 Plot a histogram for the difference image (might have to subset histogram source to area with equal amounts of landslides and non-landslides).

Create a figure to interactively view the data and set a threshold using matplotlib